# Personality Atlas — Deep Dive

**Explore any of the 44 personality models in depth.** Train classifiers, visualize embeddings, and run cross-model analysis.

> Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST. Under review.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wildertrek/survey/blob/main/notebooks/atlas_deep_dive.ipynb)

**No API keys required.** All 44 models have pre-computed embeddings and trained classifiers in the repository.

---

## 1. Setup

In [ ]:
import os
if not os.path.exists("atlas"):
    !git clone --depth 1 https://github.com/Wildertrek/survey.git atlas
else:
    print("Atlas already cloned.")

In [ ]:
!pip install -q faiss-cpu

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

import ast
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import faiss

sns.set_style("whitegrid")
print("All libraries loaded.")

## 2. Select a Model

Change `SLUG` below to explore any of the 44 models.

| Category | Models (slug) |
|----------|---------------|
| **Trait-Based** | `ocean` (Big Five), `hex` (HEXACO), `mbti`, `epm` (Eysenck), `sixteenpf`, `ftm` (Four Temperaments) |
| **Narcissism-Based** | `npi`, `pni`, `ffni`, `ffni_sf`, `narq`, `hsns`, `dtm` (Dark Triad), `dt4` (Dark Tetrad), `mcmin`, `ipn` |
| **Motivational/Value** | `stbv` (Schwartz Values), `sdt` (Self-Determination), `rft`, `aam`, `mst`, `cs` (Clifton) |
| **Cognitive/Learning** | `pct`, `scm`, `cest`, `fsls` (Felder-Silverman) |
| **Clinical/Health** | `mmpi`, `scid` (DSM), `bdi` (Depression), `gad7` (Anxiety), `wais`, `tci`, `mcmi`, `tmp`, `rit` (Rorschach), `tat` |
| **Interpersonal/Conflict** | `disc`, `tki` (Thomas-Kilmann) |
| **Application-Specific** | `riasec` (Holland Careers), `cmoa`, `tei`, `bt` (Bartle Types), `em` (Enneagram), `papc` |

In [ ]:
# ============================================
# CHANGE THIS to explore a different model
SLUG = "tci"
# ============================================

# Load dataset, embeddings, pre-trained model, and label encoder
df = pd.read_csv(f"atlas/datasets/{SLUG}.csv")
emb_df = pd.read_csv(f"atlas/Embeddings/{SLUG}_embeddings.csv")
clf = joblib.load(f"atlas/models/{SLUG}_rf_model.pkl")
le = joblib.load(f"atlas/models/{SLUG}_label_encoder.pkl")

# Parse embeddings from CSV strings to numpy array
X = np.array([ast.literal_eval(e) for e in emb_df["Embedding"]])
y = df["Factor"].values
y_encoded = le.transform(y)

print(f"Model: {SLUG.upper()}")
print(f"Traits: {len(df)}, Factors: {df['Factor'].nunique()}, Embedding dim: {X.shape[1]}")
print(f"Factors: {sorted(df['Factor'].unique())}")

## 3. Dataset Inspection

In [ ]:
# Factor distribution
factor_counts = df["Factor"].value_counts().sort_index()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
factor_counts.plot(kind="barh", ax=axes[0], color=sns.color_palette("husl", len(factor_counts)))
axes[0].set_xlabel("Trait Count")
axes[0].set_title(f"{SLUG.upper()} — Factor Distribution ({len(df)} traits)")

# Pie chart
axes[1].pie(factor_counts, labels=factor_counts.index, autopct="%1.0f%%",
            colors=sns.color_palette("husl", len(factor_counts)))
axes[1].set_title(f"{SLUG.upper()} — Factor Proportions")

plt.tight_layout()
plt.show()

In [ ]:
# Lexical schema — sample traits per factor
print(f"{SLUG.upper()}: {len(df)} traits across {df['Factor'].nunique()} factors\n")

for factor, group in df.groupby("Factor"):
    sample = group["Adjective"].unique()[:6]
    print(f"  {factor} ({len(group)} traits): {', '.join(sample)}")

## 4. Train a Random Forest Classifier

Using the pre-computed 1536-dim embeddings from `text-embedding-3-small`.

In [ ]:
# 80/20 stratified split — same parameters as original notebooks
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Train fresh RF (same hyperparameters as the pre-trained models)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc:.1%} ({sum(y_pred == y_test)}/{len(y_test)} correct)\n")

# Compare with pre-trained model from repo
y_pred_pretrained = clf.predict(X_test)
acc_pretrained = accuracy_score(y_test, y_pred_pretrained)
print(f"Pre-trained model accuracy: {acc_pretrained:.1%}")
print(f"Freshly trained accuracy:   {acc:.1%}")
if abs(acc - acc_pretrained) < 0.001:
    print("Identical — reproduced successfully.")

In [ ]:
# Full classification report
print(classification_report(y_test, y_pred, target_names=le.classes_))

## 5. Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=le.classes_, columns=le.classes_)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Raw counts
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues", ax=axes[0])
axes[0].set_xlabel("Predicted")
axes[0].set_ylabel("Actual")
axes[0].set_title(f"{SLUG.upper()} — Confusion Matrix (counts)")

# Normalized (percentages per row)
cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
cm_norm_df = pd.DataFrame(cm_norm, index=le.classes_, columns=le.classes_)
sns.heatmap(cm_norm_df, annot=True, fmt=".0%", cmap="Blues", ax=axes[1])
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Actual")
axes[1].set_title(f"{SLUG.upper()} — Confusion Matrix (row-normalized)")

plt.tight_layout()
plt.show()

## 6. Cross-Validation

In [ ]:
cv_scores = cross_val_score(
    RandomForestClassifier(n_estimators=100, random_state=42),
    X, y_encoded, cv=5, scoring="accuracy"
)

print(f"5-Fold Cross-Validation for {SLUG.upper()}:")
print(f"  Fold scores: {', '.join(f'{s:.1%}' for s in cv_scores)}")
print(f"  Mean: {cv_scores.mean():.1%} (+/- {cv_scores.std() * 2:.1%})")

# Visualize
fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(range(1, 6), cv_scores, color=sns.color_palette("husl", 5), alpha=0.8)
ax.axhline(y=cv_scores.mean(), color="red", linestyle="--", label=f"Mean = {cv_scores.mean():.1%}")
ax.set_xlabel("Fold")
ax.set_ylabel("Accuracy")
ax.set_title(f"{SLUG.upper()} — 5-Fold Cross-Validation")
ax.set_ylim(0, 1.05)
ax.legend()
plt.tight_layout()
plt.show()

## 7. PCA — Embedding Space Visualization

In [ ]:
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)
cumvar = np.cumsum(pca.explained_variance_ratio_) * 100

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Scree plot
axes[0].bar(range(1, 11), pca.explained_variance_ratio_ * 100, alpha=0.6, label="Individual")
axes[0].plot(range(1, 11), cumvar, "r-o", markersize=4, label="Cumulative")
axes[0].set_xlabel("PC")
axes[0].set_ylabel("Variance Explained (%)")
axes[0].set_title(f"{SLUG.upper()} — Scree Plot")
axes[0].legend(fontsize=8)

# PC1 vs PC2 colored by factor
colors = sns.color_palette("husl", df["Factor"].nunique())
for i, factor in enumerate(sorted(df["Factor"].unique())):
    mask = y == factor
    axes[1].scatter(X_pca[mask, 0], X_pca[mask, 1], c=[colors[i]], s=30, alpha=0.7, label=factor)
axes[1].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[1].set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
axes[1].set_title(f"{SLUG.upper()} — PC1 vs PC2")
axes[1].legend(fontsize=7, loc="best", ncol=max(1, df['Factor'].nunique() // 8 + 1))

# PC1 vs PC3
for i, factor in enumerate(sorted(df["Factor"].unique())):
    mask = y == factor
    axes[2].scatter(X_pca[mask, 0], X_pca[mask, 2], c=[colors[i]], s=30, alpha=0.7, label=factor)
axes[2].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[2].set_ylabel(f"PC3 ({pca.explained_variance_ratio_[2]*100:.1f}%)")
axes[2].set_title(f"{SLUG.upper()} — PC1 vs PC3")

plt.tight_layout()
plt.show()

print(f"\nFirst 10 PCs capture {cumvar[-1]:.1f}% of variance")

## 8. KMeans Clustering

In [ ]:
n_factors = df["Factor"].nunique()

kmeans = KMeans(n_clusters=n_factors, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# KMeans clusters in PCA space
scatter = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap="tab10", s=30, alpha=0.7)
axes[0].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[0].set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
axes[0].set_title(f"{SLUG.upper()} — KMeans ({n_factors} clusters)")
plt.colorbar(scatter, ax=axes[0], ticks=range(n_factors))

# Ground-truth factors in PCA space (for comparison)
scatter2 = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=y_encoded, cmap="tab10", s=30, alpha=0.7)
axes[1].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[1].set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
axes[1].set_title(f"{SLUG.upper()} — Ground-Truth Factors")
plt.colorbar(scatter2, ax=axes[1], ticks=range(n_factors))

plt.tight_layout()
plt.show()

# Cluster purity: how well do clusters align with factors?
from collections import Counter
purity_scores = []
for c in range(n_factors):
    mask = clusters == c
    if mask.sum() > 0:
        counts = Counter(y[mask])
        majority = counts.most_common(1)[0][1]
        purity_scores.append(majority / mask.sum())

mean_purity = np.mean(purity_scores)
print(f"Mean cluster purity: {mean_purity:.1%} (1.0 = perfect alignment with factors)")

## 9. Feature Importance — Top Embedding Dimensions

In [ ]:
importances = rf.feature_importances_
top_k = 30
top_idx = np.argsort(importances)[-top_k:][::-1]

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(range(top_k), importances[top_idx], color="steelblue", alpha=0.8)
ax.set_xticks(range(top_k))
ax.set_xticklabels([f"d{i}" for i in top_idx], rotation=45, fontsize=8)
ax.set_xlabel("Embedding Dimension")
ax.set_ylabel("Feature Importance")
ax.set_title(f"{SLUG.upper()} — Top {top_k} Most Important Embedding Dimensions")
plt.tight_layout()
plt.show()

# Cumulative importance
sorted_imp = np.sort(importances)[::-1]
cum_imp = np.cumsum(sorted_imp)
n_90 = np.searchsorted(cum_imp, 0.9) + 1
print(f"Top {n_90} of {len(importances)} dimensions capture 90% of importance")
print(f"Top 30 dimensions capture {cum_imp[29]:.1%} of importance")

## 10. Intra-Model Similarity — How Similar Are the Factors?

In [ ]:
# Compute centroid for each factor
factors_sorted = sorted(df["Factor"].unique())
centroids = np.array([X[y == f].mean(axis=0) for f in factors_sorted])

# Cosine similarity between factor centroids
norms = np.linalg.norm(centroids, axis=1, keepdims=True)
centroids_norm = centroids / norms
sim_matrix = centroids_norm @ centroids_norm.T

sim_df = pd.DataFrame(sim_matrix, index=factors_sorted, columns=factors_sorted)

fig, ax = plt.subplots(figsize=(max(8, len(factors_sorted) * 0.8), max(6, len(factors_sorted) * 0.6)))
sns.heatmap(sim_df, annot=True, fmt=".2f", cmap="RdYlBu_r", vmin=0.5, vmax=1.0,
            ax=ax, square=True)
ax.set_title(f"{SLUG.upper()} — Inter-Factor Cosine Similarity (centroids)")
plt.tight_layout()
plt.show()

# Most and least similar pairs
np.fill_diagonal(sim_matrix, 0)
max_idx = np.unravel_index(sim_matrix.argmax(), sim_matrix.shape)
min_idx = np.unravel_index(sim_matrix.argmin(), sim_matrix.shape)
np.fill_diagonal(sim_matrix, 1)

print(f"Most similar pair:  {factors_sorted[max_idx[0]]} <-> {factors_sorted[max_idx[1]]} (cos = {sim_matrix[max_idx]:.3f})")
print(f"Least similar pair: {factors_sorted[min_idx[0]]} <-> {factors_sorted[min_idx[1]]} (cos = {sim_matrix[min_idx]:.3f})")

## 11. Cross-Model Search — Find Related Traits Across All 44 Models

This is the atlas's core value: a single FAISS index over 6,694 traits from 44 models enables cross-tradition trait retrieval.

In [ ]:
# Build the full atlas FAISS index
CATEGORIES = {
    "Trait-Based": ["ocean", "hex", "mbti", "epm", "sixteenpf", "ftm"],
    "Narcissism-Based": ["npi", "pni", "hsns", "dtm", "dt4", "ffni", "ffni_sf", "narq", "mcmin", "ipn"],
    "Motivational/Value": ["stbv", "sdt", "rft", "aam", "mst", "cs"],
    "Cognitive/Learning": ["pct", "cest", "scm", "fsls"],
    "Clinical/Health": ["mmpi", "scid", "bdi", "gad7", "wais", "tci", "mcmi", "tmp", "rit", "tat"],
    "Interpersonal/Conflict": ["disc", "tki"],
    "Application-Specific": ["riasec", "cmoa", "tei", "bt", "em", "papc"]
}
slug_to_cat = {s: c for c, slugs in CATEGORIES.items() for s in slugs}

all_slugs = sorted([f.replace(".csv", "") for f in os.listdir("atlas/datasets") if f.endswith(".csv")])
all_vecs, all_labels, all_cats, all_factors, all_adjs = [], [], [], [], []

for slug in all_slugs:
    d = pd.read_csv(f"atlas/datasets/{slug}.csv")
    e = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    vecs = np.array([ast.literal_eval(v) for v in e["Embedding"]])
    all_vecs.append(vecs)
    all_labels.extend([slug.upper()] * len(d))
    all_cats.extend([slug_to_cat.get(slug, "Unknown")] * len(d))
    all_factors.extend(d["Factor"].values)
    all_adjs.extend(d["Adjective"].values)

X_all = np.vstack(all_vecs)
X_norm = (X_all / np.linalg.norm(X_all, axis=1, keepdims=True)).astype(np.float32)
index = faiss.IndexFlatIP(X_norm.shape[1])
index.add(X_norm)

print(f"FAISS index: {index.ntotal} vectors from {len(all_slugs)} models")

In [ ]:
# Search: pick a factor from the selected model
# Change QUERY_FACTOR to search for a different factor
QUERY_FACTOR = df["Factor"].value_counts().index[0]  # default: most common factor

query_mask = df["Factor"] == QUERY_FACTOR
query_idx = df[query_mask].index[0]
q = X[query_idx].reshape(1, -1).astype(np.float32)
q = q / np.linalg.norm(q)

D, I = index.search(q, 25)

query_trait = df.iloc[query_idx]
print(f"Query: {SLUG.upper()} / {QUERY_FACTOR} — \"{query_trait['Adjective']}\"\n")
print(f"{'Rank':<5} {'Model':<12} {'Factor':<30} {'Adjective':<20} {'Category':<22} {'Score':.5}")
print("-" * 95)
for rank, (i, score) in enumerate(zip(I[0], D[0]), 1):
    marker = " *" if all_labels[i] != SLUG.upper() else ""
    print(f"{rank:<5} {all_labels[i]:<12} {all_factors[i]:<30} {all_adjs[i]:<20} {all_cats[i]:<22} {score:.4f}{marker}")

# Summary
result_cats = set(all_cats[i] for i in I[0])
result_models = set(all_labels[i] for i in I[0])
cross_model = sum(1 for i in I[0] if all_labels[i] != SLUG.upper())
print(f"\n{cross_model}/25 results from other models, spanning {len(result_cats)} categories and {len(result_models)} models")
print("(* = cross-model match)")

## 12. Compare All 44 Models — Accuracy Leaderboard

In [ ]:
results = []
for slug in all_slugs:
    d = pd.read_csv(f"atlas/datasets/{slug}.csv")
    m = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    enc = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    e = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    Xm = np.array([ast.literal_eval(v) for v in e["Embedding"]])
    preds = enc.inverse_transform(m.predict(Xm))
    acc = (preds == d["Factor"].values).mean()
    results.append({
        "Model": slug.upper(), "Category": slug_to_cat.get(slug, "Unknown"),
        "Traits": len(d), "Factors": d["Factor"].nunique(), "Accuracy": acc
    })

results_df = pd.DataFrame(results).sort_values("Accuracy", ascending=True)

# Highlight the selected model
colors = ["#ff6b6b" if m == SLUG.upper() else "#4ecdc4" for m in results_df["Model"]]

fig, ax = plt.subplots(figsize=(10, max(8, len(results_df) * 0.25)))
ax.barh(range(len(results_df)), results_df["Accuracy"], color=colors, alpha=0.85)
ax.set_yticks(range(len(results_df)))
ax.set_yticklabels([f"{m} ({f}F)" for m, f in zip(results_df["Model"], results_df["Factors"])], fontsize=7)
ax.set_xlabel("Accuracy")
ax.set_title(f"Atlas Accuracy Leaderboard — {SLUG.upper()} highlighted in red")
ax.axvline(x=results_df["Accuracy"].mean(), color="gray", linestyle="--", alpha=0.5, label=f"Mean = {results_df['Accuracy'].mean():.1%}")
ax.legend()
plt.tight_layout()
plt.show()

# Stats
selected = results_df[results_df["Model"] == SLUG.upper()].iloc[0]
rank = len(results_df) - results_df.index.get_loc(results_df[results_df["Model"] == SLUG.upper()].index[0])
print(f"\n{SLUG.upper()}: {selected['Accuracy']:.1%} accuracy — rank {rank}/{len(results_df)}")
print(f"Atlas mean: {results_df['Accuracy'].mean():.1%}, median: {results_df['Accuracy'].median():.1%}")

## 13. Accuracy by Category

In [ ]:
cat_stats = pd.DataFrame(results).groupby("Category").agg(
    Models=("Model", "count"),
    Mean_Accuracy=("Accuracy", "mean"),
    Min_Accuracy=("Accuracy", "min"),
    Max_Accuracy=("Accuracy", "max"),
    Total_Traits=("Traits", "sum")
).sort_values("Mean_Accuracy", ascending=False)

cat_stats["Mean_Accuracy"] = cat_stats["Mean_Accuracy"].apply(lambda x: f"{x:.1%}")
cat_stats["Min_Accuracy"] = cat_stats["Min_Accuracy"].apply(lambda x: f"{x:.1%}")
cat_stats["Max_Accuracy"] = cat_stats["Max_Accuracy"].apply(lambda x: f"{x:.1%}")
cat_stats

## 14. 3072-dim Embedding Upgrade (Experiment 2)

Compare the selected model's accuracy using 1536-dim vs 3072-dim embeddings. The 3072-dim assets are downloaded from [Hugging Face Hub](https://huggingface.co/datasets/Wildertrek/personality-atlas-3072).

In [ ]:
!pip install -q huggingface_hub
from huggingface_hub import hf_hub_download

HF_REPO = "Wildertrek/personality-atlas-3072"

# Download 3072-dim assets for the selected model
emb3072_path = hf_hub_download(HF_REPO, f"Embeddings_3072/{SLUG}_embeddings.csv", repo_type="dataset")
model3072_path = hf_hub_download(HF_REPO, f"models_3072/{SLUG}_rf_model.pkl", repo_type="dataset")
enc3072_path = hf_hub_download(HF_REPO, f"models_3072/{SLUG}_label_encoder.pkl", repo_type="dataset")

emb3072_df = pd.read_csv(emb3072_path)
X3072 = np.array([ast.literal_eval(e) for e in emb3072_df["Embedding"]])
clf3072 = joblib.load(model3072_path)
le3072 = joblib.load(enc3072_path)

# Full-dataset accuracy comparison
df_full = pd.read_csv(f"atlas/datasets/{SLUG}.csv")
y_true = df_full["Factor"].values

preds_1536 = le.inverse_transform(clf.predict(X))
preds_3072 = le3072.inverse_transform(clf3072.predict(X3072))

acc_1536 = (preds_1536 == y_true).mean()
acc_3072 = (preds_3072 == y_true).mean()

print(f"{SLUG.upper()} accuracy comparison:")
print(f"  1536-dim (text-embedding-3-small): {acc_1536:.1%}")
print(f"  3072-dim (text-embedding-3-large): {acc_3072:.1%}")
print(f"  Delta: {(acc_3072 - acc_1536)*100:+.1f} percentage points")
print(f"  Embedding dimensions: {X.shape[1]} → {X3072.shape[1]}")

In [ ]:
# PCA comparison: 1536 vs 3072 embedding spaces
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, (Xd, dim_label) in zip(axes, [(X, "1536-dim"), (X3072, "3072-dim")]):
    pca_d = PCA(n_components=2)
    Xp = pca_d.fit_transform(Xd)
    colors_d = sns.color_palette("husl", df_full["Factor"].nunique())
    for i, factor in enumerate(sorted(df_full["Factor"].unique())):
        mask = y_true == factor
        ax.scatter(Xp[mask, 0], Xp[mask, 1], c=[colors_d[i]], s=20, alpha=0.6, label=factor)
    ax.set_xlabel(f"PC1 ({pca_d.explained_variance_ratio_[0]*100:.1f}%)")
    ax.set_ylabel(f"PC2 ({pca_d.explained_variance_ratio_[1]*100:.1f}%)")
    ax.set_title(f"{SLUG.upper()} — {dim_label}")
    if df_full["Factor"].nunique() <= 12:
        ax.legend(fontsize=6, loc="best")

plt.suptitle(f"Embedding Space Comparison: 1536-dim vs 3072-dim", fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

---

**Repository:** [github.com/Wildertrek/survey](https://github.com/Wildertrek/survey) | **3072-dim assets:** [Hugging Face Hub](https://huggingface.co/datasets/Wildertrek/personality-atlas-3072)  
**Paper:** Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST.  
**License:** MIT